## Statistical Analysis

We provide a module (`paper_ANOVA.py`) with functions used here for fitting ANOVA and ANCOVA's to data, with additional bootstrapped confidence intervals and partial eta squared $(\eta^2_p)$ effect sizes for main effects. 

The provided module requires :

    - numpy
    - pandas
    - scipy
    - statsmodels

To be available in your environment.  

DataFrames can be loaded with `pandas.read_pickle(<file path>)`:


In [1]:
import sys
import os

# Get the absolute path to the project root (one directory up from notebooks/)
root_path = os.path.abspath("..")
sys.path.append(root_path)

In [ ]:
import pandas as pd
from src import paper_ANOVA as pA

file_path = '../Data/Point_data.pkl'
df_point_data = pd.read_pickle(file_path)


### Data setup

We have provided a simple function (`stats_dataframe`) which will set up data to be used when performing analysis. This will take as input one of the datasets available, a DV column string, and optionally, and IV column string (for ANCOVA), and dtype/scaling options for both IV and DV.


In [2]:
df = pA.stats_dataframe(df_point_data, DV_col = 'Hull_volume',DV_dtype = float, DV_scale = 1, IV_col="Total_Cable", IV_scale = 0.001)
df.head(5)

,DV,Type,Subtype,IV
0,298.593973,T4,c,145.670186
1,565.808202,T5,d,222.915432
2,323.388443,T4,d,139.528669
3,583.163533,T5,c,281.277386
4,638.860559,T4,a,269.772983


### Omnibus tests (ANOVA / ANCOVA)

The `ANOVAModel` class handles models fitting. You need an input dataframe and then a formula in the style of `statsmodels.formula.api`. Type III models are fit by default, with 1000 bootstrap replicates for confidence intervals around effect sizes. The same function (through the formula syntax) works for both ANOVA and ANCOVA. Note, that with the large daatsets - analysis on all nodes or edges, this can take a long time to run.

$\eta^2_p$ effect size thresholds:

small $\geq0.01$

medium $\geq 0.06$

large $\geq0.14$

`ANOVAModel` wil always use a full factorial design.

In [7]:
# initialise model
model = pA.ANOVAModel(df, 'DV ~ C(Type) * C(Subtype) * IV', n_boot = 1000)
# fit and show summary
model.fit();
model.summary(precision = 3)

ANOVA Summary
Formula: DV ~ C(Type) * C(Subtype) * IV
N observations: 5828
Model type: Standard OLS
----------------------------------------------------------------------

                             sum_sq      df         F  PR(>F)  partial_eta2  eta2_ci_low  eta2_ci_high
Intercept              1.554047e+06     1.0   300.516   0.000         0.049        0.037         0.065
C(Type)                1.674463e+05     1.0    32.380   0.000         0.006        0.002         0.010
C(Subtype)             8.850351e+04     3.0     5.705   0.001         0.003        0.001         0.007
C(Type):C(Subtype)     2.054852e+04     3.0     1.325   0.264         0.001        0.000         0.003
IV                     2.210190e+07     1.0  4273.985   0.000         0.424        0.375         0.481
C(Type):IV             5.406811e+05     1.0   104.555   0.000         0.018        0.011         0.027
C(Subtype):IV          8.052343e+04     3.0     5.190   0.001         0.003        0.001         0.007
C(Ty

,sum_sq,df,F,PR(>F),partial_eta2,eta2_ci_low,eta2_ci_high
Intercept,1.554047e+06,1.0,300.515920,1.115276e-65,0.049164,0.037437,0.064740
C(Type),1.674463e+05,1.0,32.380151,1.329654e-08,0.005540,0.002289,0.010398
C(Subtype),8.850351e+04,3.0,5.704827,6.769925e-04,0.002936,0.001021,0.007291
C(Type):C(Subtype),2.054852e+04,3.0,1.324532,2.644402e-01,0.000683,0.000139,0.003373
IV,2.210190e+07,1.0,4273.984523,0.000000e+00,0.423755,0.375399,0.480732
C(Type):IV,5.406811e+05,1.0,104.554920,2.454988e-24,0.017672,0.011083,0.026618
C(Subtype):IV,8.052343e+04,3.0,5.190441,1.400937e-03,0.002672,0.000991,0.006646
C(Type):C(Subtype):IV,3.991073e+04,3.0,2.572596,5.232311e-02,0.001326,0.000334,0.004518
Residual,3.005539e+07,5812.0,NaN,NaN,NaN,NaN,NaN


### _Post-hoc_ comparisons.

For comparing pairwise group means `ANOVAModel` has a `pairwise_posthoc` method. To get $\mathrm{Cohen's}_d$ effect sizes and mean differences (with confidence intervals) within one categorical factor use `model.pairwise_posthoc(factor = 'Type')` for example, to compare between T4 and T5.

To look at differences within groups (Differences between subtypes within types) you can use the additional `by` parameter.

$\mathrm{Cohen's}_d$ cut off values:

small $\geq0.2$

medium $\geq 0.5$

large $\geq0.8$


In [8]:
# Compare T4 vs T5 for our current DV, Hull Volume
model.pairwise_posthoc(factor = 'Type')

,level_a,level_b,es,ci_low_es,ci_high_es,diff,ci_low_diff,ci_high_diff,n_a,n_b
0,T4,T5,0.727957,0.680168,0.77265,134.859265,125.402056,144.036167,3026,2802


In [10]:
# Compare subtypes within types for our current DV
model.pairwise_posthoc(factor = 'Subtype', by = 'Type')

,Type,level_a,level_b,es,ci_low_es,ci_high_es,diff,ci_low_diff,ci_high_diff,n_a,n_b
0,T4,a,b,0.247019,0.146513,0.353067,55.615918,33.139974,79.322227,713,722
1,T5,b,a,-0.129337,-0.225356,-0.019416,-15.759480,-27.149152,-2.323871,713,684
2,T4,c,a,-0.170370,-0.272549,-0.073002,-40.452182,-63.697028,-17.288824,825,713
3,T5,c,a,0.268442,0.164639,0.373866,32.527493,20.047926,45.343580,719,684
4,T4,c,b,0.065352,-0.033784,0.174548,15.163735,-7.768422,39.603124,825,722
5,T5,c,b,0.408855,0.301646,0.507790,48.286973,35.682919,59.454167,719,713
6,T4,c,d,0.022102,-0.072187,0.118480,5.147660,-16.646757,27.841739,825,766
7,T4,d,a,-0.201483,-0.305093,-0.105804,-45.599843,-68.061296,-23.520100,766,713
8,T5,d,a,0.145045,0.044728,0.259229,17.068941,5.197772,30.603041,686,684
9,T4,d,b,0.045427,-0.051898,0.145167,10.016075,-11.449241,32.192473,766,722


### Comparing Slopes

The full OLS regression model summary is kept in the model output, and can be show using `model.results_.model.summary()` to obtain the $\beta$ coefficient for the IV factor.

To look at individual group slopes you can obtain these using `model.simple_slopes()`. Finally, to look at slope differences, you can use `model.simple_slopes_pairwise()`:

In [11]:
# full OLS Regression output
model.results_.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     DV   R-squared:                       0.867
Model:                            OLS   Adj. R-squared:                  0.867
Method:                 Least Squares   F-statistic:                     1625.
Date:                 Di, 09 Dez 2025   Prob (F-statistic):               0.00
Time:                        18:05:30   Log-Likelihood:                -33179.
No. Observations:                5828   AIC:                         6.639e+04
Df Residuals:                    5812   BIC:                         6.650e+04
Df Model:                          15                                         
Covariance Type:                  HC3                                         
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                         -183.3949     10.579    -17.335      0.000    -204.130    -162.660
C(Type)[T.T5]                       82.7199     14.537      5.690      0.000      54.228     111.212
C(Subtype)[T.b]                     38.7083     14.857      2.605      0.009       9.589      67.828
C(Subtype)[T.c]                     52.0066     14.272      3.644      0.000      24.035      79.979
C(Subtype)[T.d]                     10.6232     14.758      0.720      0.472     -18.302      39.549
C(Type)[T.T5]:C(Subtype)[T.b]      -35.3912     21.159     -1.673      0.094     -76.863       6.080
C(Type)[T.T5]:C(Subtype)[T.c]      -13.7995     20.501     -0.673      0.501     -53.980      26.381
C(Type)[T.T5]:C(Subtype)[T.d]        3.8168     21.672      0.176      0.860     -38.659      46.293
IV                                   3.1178      0.048     65.376      0.000       3.024       3.211
C(Type)[T.T5]:IV                    -0.7060      0.069    -10.225      0.000      -0.841      -0.571
C(Subtype)[T.b]:IV                  -0.0701      0.072     -0.977      0.328      -0.211       0.070
C(Subtype)[T.c]:IV                  -0.1249      0.069     -1.815      0.070      -0.260       0.010
C(Subtype)[T.d]:IV                   0.1506      0.072      2.097      0.036       0.010       0.291
C(Type)[T.T5]:C(Subtype)[T.b]:IV     0.1534      0.106      1.440      0.150      -0.055       0.362
C(Type)[T.T5]:C(Subtype)[T.c]:IV    -0.0316      0.101     -0.314      0.754      -0.229       0.166
C(Type)[T.T5]:C(Subtype)[T.d]:IV    -0.1646      0.107     -1.535      0.125      -0.375       0.046
==============================================================================
Omnibus:                     1423.850   Durbin-Watson:                   2.007
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9244.349
Skew:                           1.007   Prob(JB):                         0.00
Kurtosis:                       8.832   Cond. No.                     1.06e+04
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The condition number is large, 1.06e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
### Slopes for IV (Total Cable length here) for each 
# factor level of Type (T4, T5)
model.simple_slopes('IV','Type')

,Type,slope,slope_std,se,ci_low,ci_high,t,p,n
0,T4,3.063620,0.932096,0.025420,3.013778,3.113462,120.520128,0.0,3026
1,T5,2.362869,0.865173,0.027902,2.308157,2.417580,84.683346,0.0,2802


In [12]:
# pairwise differences between slopes for levels of the 
# factor Type (T4, T5)
model.simple_slopes_pairwise('IV','Type')

,Type_a,Type_b,slope_a,slope_b,diff,se_boot,ci_low,ci_high,p_boot,n_a,n_b
0,T4,T5,3.06362,2.362869,0.700751,0.036799,0.632068,0.772663,0.0,3026,2802
